## Qualtiy of neural control reflects local gradient alignment

Here we perform three experiments:
1. Gradient alignment vs. standard linear regression EV as a function of sample size
2. Original strech control objective vs. gradient alignment EV from linear mapping
3. Targeted control objective vs. gradient alignment EV. 

The goal of #1 is to see whether better pointwise function alignment leads to better gradient alignment.
If this is not the case, then this would explan #2. Given these observations we improve upon the control metric's sensitivity to gradient with #3.

In [1]:
import torch
import torchvision.models as models
from util.modelregressions import CNNCrossFit
from util.misc_functions import float2rgb
from dataset.hvm import HVMDataset
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# load up target and source models
squeezenet_target = models.squeezenet1_0(pretrained=True).features
squeezenet_source = models.squeezenet1_1(pretrained=True).features
# define basic params 
cnn_layer = 7
source_units = np.arange(128)
target_units = np.arange(128)
img_dim = [256,256,3]
# pre-load data into memory for speed
hvmdataset = HVMDataset(5500)

In [3]:
# create a control experiment (squeezenet1_1-->squeezenet1_0)
snet2snet200 = CNNCrossFit(squeezenet_target,squeezenet_source,cnn_layer,target_units,source_units)
# extract layer activation across two models
snet2snet200.design(hvmdataset,5000,500,record_gradient = True)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000002558C4D61E0> (for post_execute):


ValueError: Floating point image RGB values must be in the 0..1 range.

ValueError: Floating point image RGB values must be in the 0..1 range.

In [ ]:
# linear prediction of target unit, record performance
snet2snet200.fit(n_train = 500,n_test = 500)
# implement source control of target units, record performance
snet2snet200.control()

In [ ]:
trainloader = torch.utils.data.DataLoader(hvmdataset, batch_size=2,
                        shuffle=False, num_workers=0)
x = iter(trainloader).next()
x.requires_grad = True
y = squeezenet_target[:6].forward(x-.502)
y2 = squeezenet_source[:6].forward(x-.502)
ytar = y[1,4].mean()
ytar.backward()
xg = x.grad[1].permute(1,2,0).detach().numpy()
plt.subplot(1,2,1)
plt.imshow(float2rgb(xg))
plt.subplot(1,2,2)
plt.imshow(x[1].permute(1,2,0).detach().numpy())
plt.show()

In [ ]:
x[0,2,0,1]-.5